In [180]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import yaml

def read_yaml(path):
    with open(path, 'rb') as yaml_file:
        return yaml.safe_load(yaml_file)


In [181]:
df = pd.read_csv("../dataset/UCI_Credit_Card.csv")
df.rename(columns={'PAY_0': 'PAY_1', 'SEX': 'GENDER', 'default.payment.next.month': 'DEFAULTS'}, inplace=True)
df.drop(columns=['ID'], axis=1, inplace=True)
df.head().T

,0,1,2,3,4
LIMIT_BAL,20000.0,120000.0,90000.0,50000.0,50000.0
GENDER,2.0,2.0,2.0,2.0,1.0
EDUCATION,2.0,2.0,2.0,2.0,2.0
MARRIAGE,1.0,2.0,2.0,1.0,1.0
AGE,24.0,26.0,34.0,37.0,57.0
PAY_1,2.0,-1.0,0.0,0.0,-1.0
PAY_2,2.0,2.0,0.0,0.0,0.0
PAY_3,-1.0,0.0,0.0,0.0,-1.0
PAY_4,-1.0,0.0,0.0,0.0,0.0
PAY_5,-2.0,0.0,0.0,0.0,0.0


In [182]:
df.EDUCATION.value_counts()

2    14030
1    10585
3     4917
5      280
4      123
6       51
0       14
Name: EDUCATION, dtype: int64

In [183]:
config_data = read_yaml("../config/schema.yaml")
cat_col = config_data.get("domain_value")
# if not cat_col.get("MARRIAGE") in df.MARRIAGE.unique() and not cat_col.get("EDUCATION") in df.EDUCATION.unique():
#         df["MARRIAGE"].replace(to_replace=0, value=3, inplace=True)
#         df["EDUCATION"].replace(to_replace=[0, 5, 6], value=4, inplace=True)

In [184]:
cat_col.get("MARRIAGE") in df.MARRIAGE.unique() and cat_col.get("EDUCATION") in df.EDUCATION.unique()

<ipython-input-184-4e205918d405>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  cat_col.get("MARRIAGE") in df.MARRIAGE.unique() and cat_col.get("EDUCATION") in df.EDUCATION.unique()


False

In [185]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=["DEFAULTS"], axis=1)
y = df["DEFAULTS"]
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.25)

In [186]:
for df in (train_x, test_x):
    if not cat_col.get("MARRIAGE") in df.MARRIAGE.unique() and not cat_col.get("EDUCATION") in df.EDUCATION.unique():
            df["MARRIAGE"].replace(to_replace=0, value=3, inplace=True)
            df["EDUCATION"].replace(to_replace=[0, 5, 6], value=4, inplace=True)
            print(True)
    else:
        print(False)

True
True


<ipython-input-186-10897d1ee964>:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if not cat_col.get("MARRIAGE") in df.MARRIAGE.unique() and not cat_col.get("EDUCATION") in df.EDUCATION.unique():


In [188]:
train_x.EDUCATION.value_counts()

2    10497
1     7950
3     3696
4      357
Name: EDUCATION, dtype: int64

In [1]:
from urllib.request import urlretrieve
import os
url = 'https://github.com/prathmesh-jagtap/Credit-Card-Default-Prediction-ML-Project/raw/main/dataset/UCI_Credit_Card_Defaults.zip'
tgz_download_dir = "tgz-data"
os.makedirs(tgz_download_dir, exist_ok=True)
defaults_file_name = os.path.basename(url)
tgz_file_path = os.path.join(tgz_download_dir, defaults_file_name)
urlretrieve(url, tgz_file_path)

('tgz-data\\UCI_Credit_Card_Defaults.zip',
 <http.client.HTTPMessage at 0x727f5f8>)

In [8]:
from zipfile import ZipFile

raw_data_dir = "raw-data"

os.makedirs(raw_data_dir, exist_ok=True)

with ZipFile('tgz-data\\UCI_Credit_Card_Defaults.zip', mode='r') as default_zip_file_obj:
    default_zip_file_obj.extractall(path=raw_data_dir)